# DeepLearning을 적용한 Q-Learning

## 표형식 표현(Tubular Expression)의 문제점 - 딥러닝 적용 이유

심층강화학습이란, 강화학습에 딥러닝을 적용한 것이다. 일반적인 강화학습에서 사용하는 표형식 표현은 각 행이 Agent의 상태, 각 열이 Agent가 취할 수 있는 행동에 해당한다. 그리고 셀의 각 값은 행동가치 Q(s, a)이다. 

Agent의 상태란 MazeTask 에서는 Agent가 있는 칸의 위치를 의미하고, CartPole Task에서는 네 개의 변수를 각각 이산변수로 변환하고 이를 다시 양자화된 값으로 변환한 값이다. 그리고 행동가치 Q(s(t), a(t))는 시각 t의 상태가 s(t)일 때 행동 a(t)를 취했을 때 얻을 수 있는 할인총보상을 의미한다.

표형식 표현을 사용하는 Q-learning의 문제점은 상태변수의 종류가 늘어나거나 각 변수를 이산변수로 변환할 때 구간을 세세하게 분할하거나 다른 작업을 실행하면 테이블의 행의 수가 급격하게 늘어난다는 점이다. 예를 들어, 입력값을 이미지의 형태로 받는다면 상태변수는 이미지의 각 픽셀이 된다. x축과 y축의 각 픽셀 수가 50픽셀이라면, 전체 상태변수는 2,500개의 상태변수로 이루어진다. 행의 수가 많은 표형식 표현으로 제대로 강화학습을 진행하려면 epoch가 매우 커야한다. 따라서 상태변수가 큰 Task는 표형식 표현으로 학습하기가 상당히 까다롭다.

## 심층강화학습 알고리즘 DQN

상태변수가 많은 Task에 강화학습을 적용하기 위해서는 행동가치 함수를 표형식으로 나타내는 대신에, layer가 깊은 신경망으로 바꾸어 나타낸다. 다시 말해, 각 상태변수의 값을 신경망의 입력값으로 받는다. 따라서 신경망의 입력층이 갖는 유닛 수는 상태변수의 수와 같게 된다. CartPole Task를 예로 들면, 상태는 위치, 속도, 각도, 각속도 네 개의 변수이므로 입력 유닛은 네 개가 된다. 

#### 신경망에 상태변수를 입력할 때는 이산변수로 변환하는 과정이 필요없다. 

출력층 유닛 수는 행동의 가짓수와 같다. CartPole Task로 예를 들면, 왼쪽 또는 오른쪽으로 미는 두 가지 방법이 있으므로 출력층 유닛 수는 두 개가 된다.

출력층 유닛의 출력값은 행동가치 함수 Q(s(t), a(t))의 값이다. 즉, 해당 행동을 취했을 때 받을 수 있는 할인총보상값을 출력하는 것이다. 그리고 출력층 각 유닛이 출력하는 할인총보상값을 비교해보고 행동을 결정한다. MNIST 손글씨 분류 같은 Clasification 문제가 아니라 구체적인 값을 예측하는 Regression 문제에 해당한다.

행동가치 함수를 출력하는 신경망을 구현할 때 신경망의 결합 가중치를 학습시키는 방법은 역전파 계산에서 어떤 특정 오차함수를 사용하면서 오차를 어떻게 계산하는지가 중요하다. 일반적인 Q-learning에서 행동가치 함수 Q를 수정하는 식은 다음과 같다

#### Q-learning: Q(s, a, t) = Q(s, a, t) + eta [R(t+1) + gamma_max_Q(s+1, a+1, t) - Q(s, a, t)]

현재의 수정식을 사용하는 이유는 Q(s, a, t) = R(t+1) + gamma_max_Q(s+1, a+1, t)와 같은 식이 성립되기를 희망하기 때문이다.

따라서 특정 시각 t에서 상태가 s(t)였을 때 행동 a(t)를 취한다면 출력값은 Q(s(t), a(t))가 되고, 이 출력값 Q(s(t), a(t))가 R(t+1) + gamma_max Q(s+1, a+1, t)에 가까워지도록 학습을 수행하면 된다. 출력값과 원하는 값의 차이를 계산하고 차이의 제곱을 오차로 사용한다.(제곱오차함수)

#### Squared Error: E(s(t), a(t)) = {R(t+1) + gamma_max_Q(s+1, a+1, t) - Q(s, a, t)} ** 2

이때 상태 s(t+1)은 s(t)에서 행동 a(t)를 취해보고 알아내고, max_Q(s(t+1), a)의 값은 신경망에 상태 s(t+1)을 입력해서 알아낸다.

## DQN 구현 시 중요한 네 가지 기법 및 다섯 가지 주의점

#### 중요한 네 가지 기법

* 1. Experience Replay
    일반적인 Q-learning에서는 표형식처럼 각 단계마다 해당 단계의 내용(experience)을 학습한다. 하지만 DQN은 메모리에 각 단계의 내용을 저장해 둔 다음 무작위로 내용을 꺼내(replay) 신경망을 학습시킨다. 이때 이 한 단계의 내용을 transition이라고 한다.
    각 단계마다 해당 단계의 내용을 학습하면 신경망이 시간적으로 상관관계가 높은 애용을 연속적으로 학습하게 되므로 결합 가중치 학습이 안정적으로 진행되기 어려워진다. Experience Replay는 이러한 문제를 해결하기 위한 대책이다.


* 2. Fixed Target Q-Network
    행동을 결정하는 main-network와 오차함수 계산 시에 행동가치를 계산하는 target-network라는 두 가지 신경망을 사용한다. DQN에서는 가치함수 Q(s, a)를 수정하려고 하지만 Q-learning 알고리즘으로 행동가치 함수를 수정하려면 다음 시각의 상태 s(t+1)에서의 가치함수 Q(s(t+1), a)가 필요하다. 즉, Q-function을 수정하기 위해 같은 Q-function을 사용해야 한다.하지만 같은 Q-function을 사용하면 Q-function의 학습이 불안정하기 때문에 수정에 사용되는 max_Q(s(t+1), a)를 계산할 때 이전 시각의 또 다른 Q-function(Fixed Target Q-Network)를 사용해 계산한다.


* 3. Reward Clipping
    보상 클리핑은 각 단계에서 받은 보상을 -1, 0, 1의 세 가지 값 중 하나로 고정하는 방법이다. 결과적으로 Task의 보상 설계와 상관없이 같은 하이퍼 파라미터로 DQN을 적용할 수 있다는 장점이 있다.


* 4. Huber-function
    제곱오차함수 대신 Huber function으로 오차를 계산하게 된다면 [-1, 1] 구간에서는 제곱오차와 비슷하지만 -1 이하 혹은 1 이상 구간에서는 오차의 절댓값을 함수값으로 계산한다. 오차가 큰 경우에 제곱오차를 사용하면 오차함수의 출력이 지나치게 커져서 학습이 불안정해지기 쉽기 때문에 Huber-function을 사용하여 해당 문제를 해결한다.
    
    

#### 다섯 가지 주의점

* 1. mini-batch 사용
    Experience Replay 및 유사 Fixed Target Q-Network를 구현하기 위해 mini-batch 학습을 적용한다. DQN은 각 단계의 transition(상태 s(t), 행동 a(t), 다음상태 s(t+1), 보상 r(t+1))을 메모리 객체에 저장한다. mini-batch는 이 메모리 객체에서 여러 단계 분량에 해당하는 transition을 무작위로 꺼낸 것을 가리키는 단어이다. CartPole Task의 경우 200단계를 버텨내면 게임이 종료되므로 그 다음 단계의 상태 s(t+1)이 존재하지 않기 때문에 다음 상태의 존재 여부에 따라 분기하도록 구현해야 한다.
    

* 2. mini-batch를 다루는 방법


* 3. Variable Data Type
    OpenAI Gym의 CartPole Task는 Pytorch 신경망과 서로 데이터를 주고받아야 한다. CartPole Task에서는 Numpy Type으로 다루는 반면, Pytorch는 Torch.Tensor Type으로된 Tensor를 다루기 때문에 Numpy Type과 Tensor Type 간의 변환이 잦다.


* 4. Variable Size
    Torch.Tensor의 크기에서, size 1과 size 1 x 1은 물리적으로는 같은 크기지만 의미상으로는 다른 Type이기 때문에 편의상 변환이 필요한 경우가 많다.
    

* 5. namedtuple 사용
    namedtuple을 사용하면 CartPole에서 관측된 상태 변수 값에 이름을 붙여 저장할 수 있다.